# Introduction
Welcome to the [VinBigData Chest X-ray Abnormalities Detection](https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/data) compedition.　　

このコンペティションは、胸部X線写真を15種類に分類するものです。    
15種類のうち1つは正常画像なので、残り14種類の異常所見が含まれることになります。  
これら14の異常所見に関して簡単に解説を英語で作成しましたが、日本でも作成することにしました。
（正直英語は苦手なので、なぜ最初から日本語にしなかったのかと言われると、、、）
英語版はこちら→https://www.kaggle.com/sakuraandblackcat/chest-x-ray-knowledges-for-the-14-abnormalities

このノートブックのコーディング部分は以下のノートを参考にしました。ありがとうございます。 https://www.kaggle.com/drcapa/chest-x-ray-starter  

過去のKaggleコンペでも胸部X線写真を対象にしたものがあり、とても良い解説Notebookがありますので、そちらも参考になると思います（英語オンリーです）
https://www.kaggle.com/zahaviguy/what-are-lung-opacities

Kaggleは初心者、英語も初心者ですが、日本語はネイティブ並みです(笑)  
ちなみに私自身は一般内科医ですので胸部X線写真を見る機会は多いですが、放射線科医師に比べると大雑把な知識しかありませんので、ご了承ください(^^)/


# X線写真の基礎知識
この辺は知っている人が多いかと思いますが、念のため。  
X線の吸収の度合いは、X線の波長、被写体の厚さ、被写体の密度、被写体を構成する原子によって決まりまります。生体にX線を照射すると、その吸収度によって異なる濃度（黒さの度合い）がX線画像に反映されます。
1. 骨はX線を吸収して透過させないため、人体のフィルムの中で最も白く見える部分になります。
2. 肺野は空気を多く含んでいるため比較的黒く見えます。
3. 肺に炎症があって滲出物(水に近い透過性)が多くなると白い影となります。
4. 気胸のように肺が虚脱して肺組織が縮んでしまうと、その部分は黒い影になります。  

**注）このコンペ画像には、標準画像の他に白黒反転画像も含まれています。白黒反転画像は医師が異常を見つけに行く際に好んで使う場合があります。上記の白い・黒いが逆転しますのでご注意ください！


# Table of Contents
# 目次

上述したように、このNotebookでは分類すべき15種類（14の異常所見含む）を簡単に解説いたします。  
基本的に、簡潔な説明を優先しますので、医学的な正確さは多少犠牲になっております。  
皆様からのコメントお待ちしております。  

- [1. No finding](#1.-No-finding)
- [2. Aortic enlargement](#2.-Aortic-enlargement)
- [3. Atelectasis](#3.-Atelectasis)
- [4. Calcification](#4.-Calcification)
- [5. Cardiomegaly](#5.-Cardiomegaly)
- [6. Consolidation](#6.-Consolidation)
- [7. ILD](#7.-ILD)
- [8. Infiltration](#8.-Infiltration)
- [9. Lung Opacity](#9.-Lung-Opacity)
- [10. Nodule/Mass](#10.-Nodule/Mass)
- [11. Other lesion](#11.-Other-lesion)
- [12. Pleural effusion](#12.-Pleural-effusion)
- [13. Pleural thickening](#13.-Pleural-thickening)
- [14. Pneumothorax](#14.-Pneumothorax)
- [15. Plumonary fibrosis](#15.-Plumonary-fibrosis)

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pydicom as dicom
import cv2

import warnings
warnings.filterwarnings("ignore")

In [ ]:
path = '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/'
#os.listdir(path)

In [ ]:
train_data = pd.read_csv(path+'train.csv')
samp_subm = pd.read_csv(path+'sample_submission.csv')

In [ ]:
#print('Number train samples:', len(train_data.index))
#print('Number test samples:', len(samp_subm.index))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4))
x = train_data['class_name'].value_counts().keys()
y = train_data['class_name'].value_counts().values
ax.bar(x, y)
ax.set_xticklabels(x, rotation=90)
ax.set_title('Distribution of the labels')
plt.grid()
plt.show()

# Show Examples

In [ ]:
def plot_example(idx_list):
    fig, axs = plt.subplots(1, 3, figsize=(15, 10))
    fig.subplots_adjust(hspace = .1, wspace=.1)
    axs = axs.ravel()
    for i in range(3):
        image_id = train_data.loc[idx_list[i], 'image_id']
        data_file = dicom.dcmread(path+'train/'+image_id+'.dicom')
        img = data_file.pixel_array
        axs[i].imshow(img, cmap='gray')
        axs[i].set_title(train_data.loc[idx_list[i], 'class_name'])
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])
        if train_data.loc[idx_list[i], 'class_name'] != 'No finding':
            bbox = [train_data.loc[idx_list[i], 'x_min'],
                    train_data.loc[idx_list[i], 'y_min'],
                    train_data.loc[idx_list[i], 'x_max'],
                    train_data.loc[idx_list[i], 'y_max']]
            p = matplotlib.patches.Rectangle((bbox[0], bbox[1]),
                                             bbox[2]-bbox[0],
                                             bbox[3]-bbox[1],
                                             ec='r', fc='none', lw=2.)
            axs[i].add_patch(p)
            

# 1. No finding
# 正常画像
正常画像になります。
基本的にはこの正常画像との差異を取って異常所見を見分けていくことになります。  
研修医（または修行中）の頃に、沢山の正常画像を読むことで、なんとなく異常所見が目に入るようになってきます。  
[Back to top](#Table-of-Contents)



In [ ]:
idx_list = train_data[train_data['class_id']==14][0:3].index.values
plot_example(idx_list)

# 2. Aortic enlargement
# 大動脈拡大  

大動脈拡大はその名の通り、大動脈が大きくなった画像で、大動脈瘤（またはその前段階）を表す所見です。  
大動脈は心臓から出て、まずは頭側に向かい脳に血液を送る分枝を出し（上行大動脈）、その後に反転して下肢に向かいます（下行大動脈）。  

大動脈拡大は上行大動脈に起こりやすいです。
一般的に大動脈瘤というと上行大動脈で直径5㎝超え、下行大動脈で直径4㎝超えを指します。  
[Back to top](#Table-of-Contents)



In [ ]:
idx_list = train_data[train_data['class_id']==0][9:12].index.values
plot_example(idx_list)

# 3. Atelectasis
# 無気肺


無気肺症とは、肺の一部または全部に空気がない状態のことです。そして、肺がつぶれている状態です。無気肺症の一般的な原因は、気管支の閉塞です。  

無気肺症では、胸部X線写真上で密度の増加が認められます（通常は白色、白黒反転画像では黒色）。  
無気肺の程度が大きいと縦隔偏位（心臓が無気肺側に引っ張られる）が起こります。

[Back to top](#Table-of-Contents)

 

In [ ]:
idx_list = train_data[train_data['class_id']==1][0:3].index.values
plot_example(idx_list)

# 4. Calcification
# 石灰化

多くの疾患や状態が胸部X線上で石灰化（カルシウム沈着）を引き起こします。  
肺や胸膜の炎症が治った部分にカルシウムが沈着する事で起こるのが一般的です。また、動脈硬化が原因で大動脈にカルシウムが沈着することもあります。また、縦隔リンパ節に石灰化が生じることもあります。 

画像では骨に似た密度（つまり高い濃度：通常画像では白い）が特徴的です。

[Back to top](#Table-of-Contents)



In [ ]:
idx_list = train_data[train_data['class_id']==2][0:3].index.values
plot_example(idx_list)

# 5. Cardiomegaly
# 心拡大、心肥大

心臓が大きくなった状態。  
心肥大の原因には、高血圧、冠動脈疾患、感染症、遺伝性疾患、心筋症など様々な疾患があります。 

心肥大症は通常、胸部の幅に対する心臓の幅の比率が50％以上の場合に診断されます。この診断基準は、本大会では画像分類の根拠になるかもしれません。 

*読者の皆様からのコメント  
**心肥大症の診断基準である心肺比＞0.5について。 
この基準は、患者が立っている状態（立位）でX線検査を行った場合にのみ有効です。患者さんが座っていたり、ベッドで寝ている場合は、この基準は使用できないです。
画像所見で、胃の中の空気が見えなければ、患者さんは寝ている（臥位）の可能性が高いです。**  

[Back to top](#Table-of-Contents)


In [ ]:
idx_list = train_data[train_data['class_id']==3][0:3].index.values
plot_example(idx_list)

# 6. Consolidation
# コンソリデーション（浸潤影）

consolidationは、日本語でもコンソリデーションと言われることが多いです。正式にはair space consolidationと呼ぶようです。肺胞内の空気を含む空間に代わって体液や細胞、組織などが浸潤して肺の透過性が低下することをいいます。

X線写真では、肺野の密度が高く（＝白く）なります。肺の血管は見えませんが、気管支には空気が残っているため、X線を吸収せずに黒く見え、正常な肺野とは白と黒が逆転します。 白地に黒い気管支が見えることから "気管支透亮像（air bronchogram）"と呼ばれています。  
＜個人定意見＞
後に出てくるinfiltration（浸潤影）やopacity（不透明）との区別は相当難しいと思います。本コンペの画像を見ても、ラベルを付ける人によってconsolidationにするかinfiltrationにするかは変わっていそうです。（勿論、私には区別がつかないというだけかもしれません）  

[Back to top](#Table-of-Contents)


In [ ]:
idx_list = train_data[train_data['class_id']==4][0:3].index.values
plot_example(idx_list)

# 7. ILD
# 間質性肺疾患

ILDは "Interstitial Lung Disease: 間質性肺疾患"の略です。 
間質性肺疾患とは、肺の間質が傷害される疾患の総称です。  
間質とは、肺胞（肺の中の空気嚢）の周囲にある薄い壁の中で、血管や小気道の周りの空間を指します。（肺胞から酸素を血管内に取り込む過程において、肺胞と血管の間にある空間と考えていただければ良いと思います）

胸部X線写真では、すりガラス様陰影（＝霞んだ不透明の領域）、線状網状影、粒状影などがある。 

[Back to top](#Table-of-Contents)
  

In [ ]:
idx_list = train_data[train_data['class_id']==5][0:3].index.values
plot_example(idx_list)

# 8. Infiltration
# 浸潤影

肺胞内に何らかの液体成分が浸潤することで、浸潤陰が生じる。 

コンソリデーションとの区別が難しく、場合によっては区別がつかないこともあります。  
参考→ https://allnurses.com/consolidation-vs-infiltrate-vs-opacity-t483538/.  
特徴はコンソリデーションとほぼ同じになります、、、

＜個人定意見＞   
consolidation（コンソリデーション）、infiltration（浸潤影）、opacity（不透明）の区別は相当難しいと思います。本コンペの画像を見ても、ラベルを付ける人によって上記どれに分類するかは変わっていそうな気がします（勿論、私には区別がつかないというだけかもしれません）。  
[Back to top](#Table-of-Contents)

 


In [ ]:
idx_list = train_data[train_data['class_id']==6][0:3].index.values
plot_example(idx_list)

# 9. Lung Opacity
# 肺野透過性低下

肺野の透過性低下とは、胸部X線写真で肺野は通常空気が多く黒いはずのところが、それよりも白くなっている領域の事です。  
あれ、それだけであれば、浸潤影もコンソリデーションも該当するのでは？と思った人は、、、正解です。
☆このLung opacityという用語はかなり定義の緩い用語だと思います☆
https://www.kaggle.com/zahaviguy/what-are-lung-opacities を参照のこと（英語）。

今までも述べてきましたが、Infiltration（浸潤）やコンソリデーションとの区別が難しく、場合によっては区別がつかないこともあります。
https://allnurses.com/consolidation-vs-infiltrate-vs-opacity-t483538/ を参照してください。  
繰り返しになりますが、ラベルを付ける人によって上記どれに分類するかは変わっていそうな気がします（勿論、私には区別がつかないというだけかもしれません）。

[Back to top](#Table-of-Contents)



In [ ]:
idx_list = train_data[train_data['class_id']==7][0:3].index.values
plot_example(idx_list)

# 10. Nodule/Mass
# 結節影/腫瘤

結節影/腫瘤とは、胸部X線画像に現れる丸い陰影（通常は直径3cm以下の円形～楕円形の白い影）のことです。  
原発性肺がん、大腸がんや腎臓がんなどの体の他の部位からの転移、結核、肺真菌症、非結核性真菌症、肺炎、肥満性肺炎、良性腫瘍などで見られます。

[Back to top](#Table-of-Contents)


In [ ]:
idx_list = train_data[train_data['class_id']==8][0:3].index.values
plot_example(idx_list)

# 11. Other lesion
# その他

その他には、他に該当しない全ての異常が含まれると思います。（なぜ、最後ではなくclass_idの10番目（正常を1番目とすると11番目）にその他が来ているのかはよくわかりません）  

具体的には、皮下気腫、骨折、骨透亮像などが含まれていそうです。

[Back to top](#Table-of-Contents)



In [ ]:
idx_list = train_data[train_data['class_id']==9][0:3].index.values
plot_example(idx_list)

# 12. Pleural effusion
# 胸水

胸水とは、胸腔内・肺外に水が溜まることです。胸腔内にたまった水の中に肺が浮いている状態をイメージすると分かり易いかもしれません。  
もう少し細かく説明します。肺は胸壁に囲まれた胸腔内にありますが、その肺の外側は、胸膜と呼ばれる薄い膜が覆っております。この胸膜は、胸壁側の層（壁側胸膜）と肺側の層（臓側胸膜）の2つの層で構成されています。これらの層の間に溜まった体液を胸水と呼びます。  
分かりにくい方は、「胸腔内にたまった水の中に肺が浮いている状態」で良いと思います。

胸水の所見は多岐にわたり、また、X線撮影が立位か仰臥位かによっても異なりますが、 
一般的な所見としては以下のような特徴があげられます。  
**-片側の横隔膜挙上、  
-横隔膜の平坦化、  
-肋骨と横隔膜の間の角度（costo-phrenic angle、肋横隔膜角）の鈍化（通常は30度以上）**


[Back to top](#Table-of-Contents)


In [ ]:
idx_list = train_data[train_data['class_id']==10][0:3].index.values
plot_example(idx_list)

# 13. Pleural thickening
# 胸膜肥厚

胸膜とは肺を覆っている膜のことで、胸膜の厚くなることを、胸膜肥厚といいます。肺野の最上部（肺尖部）に多く見られます。 

"胸膜肥厚は、典型的には肺尖部に起こり、帽子を被っているように見えることから「pulmonary apical cap：肺尖帽」と呼ばれている。胸部X線写真では、apical capは肺尖端に位置する不規則な密度として表れ、通常、幅は5mm未満である。" 
*Saito, A. et al. Pleural thickening on screening chest X-rays: a single institutional study. Respir Res 20, 138 (2019). https://doi.org/10.1186/s12931-019-1116-9*

[Back to top](#Table-of-Contents)


In [ ]:
idx_list = train_data[train_data['class_id']==11][0:3].index.values
plot_example(idx_list)

# 14. Pneumothorax
# 気胸

気胸とは、肺に穴が開いて、空気が肺から胸腔内に漏れた状態の事です。空気が漏れて胸腔内に溜まると、胸壁や肋骨があるために、漏れた空気は外に逃げることができず、結果として肺を圧迫することになります。つまり、肺は漏れた空気に押されて小さくつぶれてしまいます。  
気胸とは「肺から空気が胸腔内に漏れて、肺が小さくつぶれた状態」といえます。 

気胸の胸部レントゲン写真では、つぶれた肺は正常時よりも白く、肺が無くなって空気に置き換わった部分は一様に黒くなります（つまり肺が無くなるので血管陰影も見えなくなる）。また、肺の辺縁が線状に見えます。

[Back to top](#Table-of-Contents)


In [ ]:
idx_list = train_data[train_data['class_id']==12][1:4].index.values
plot_example(idx_list)

# 15. Plumonary fibrosis
# 肺線維症（肺線維化）

肺の線維化とは、様々な原因で肺組織が瘢痕化・線維化してしまう状態の事です。
一般的には肺の間質に炎症が起こり、その結果、壁が厚く硬くなっていき、最終的に線維化が生じますが、原因不明の特発性というものもあります。 

線維化した部分は空気の量が失われ、その結果、密な紐状陰影や顆粒状陰影が生じます。

[Back to top](#Table-of-Contents)


In [ ]:
idx_list = train_data[train_data['class_id']==13][3:6].index.values
plot_example(idx_list)

## 最後までお読みいただきありがとうございます
(・∀・)ｲｲﾈ!!　お願いします